In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import time
from itertools import islice, product
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.cross_validation import KFold

/Users/neilgetty/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


###Data Preprocessing functions

In [2]:
def window(seq, k=3):
    """ 
    Generate generate all kmers for a sequence
    Params:
        seq....dna sequence
        k......length of kmer
    """
    it = iter(seq)
    result = tuple(islice(it, k))
    if len(result) == k:
        yield result
    for elem in it:
        result = result[1:] + (elem,)
        yield result


def reverse_complement(kmer):
    """
    Generate a kmers complement
    Params:
        kmer....The kmer to generate the complement for
    """
    comp = {'a': 't', 'c': 'g', 't': 'a', 'g': 'c'}
    rc = ()
    for x in range(len(kmer)):
        rc = rc + (comp[kmer[len(kmer)-x-1]],)
    return rc


def gen_vocab(k=3):
    """
    Generate index kmer pairs for all possible kmers, binning complements together
    Params:
        k....length of kmer
    """
    all_kmers = list(product('atcg', repeat=k))
    vocab = {}
    for mer in all_kmers:
        rc = reverse_complement(mer)
        if rc in vocab:
            vocab[mer] = vocab[rc]
        else:
            vocab[mer] = len(vocab)
    return vocab


def convert_labels(labels):
    """ 
    Convert labels to indexes
    Params:
        labels...Original k class string labels
    """
    label_idxs = {}
    new_labels = np.empty(len(labels))
    for x in range(len(labels)):
        new_labels[x] = label_idxs.setdefault(labels[x], len(label_idxs))
    return new_labels


def featurize_data(file, k=3):
    """ 
    Featurize sequences and index labels
    Params:
        file....Delimited data file
    """
    data = pd.read_csv(file, names=["label", "dna"], usecols=[0,7], delimiter = '\t')
    labels = convert_labels(data.label)
    #labels = data.label
    kmers = [Counter(list(window(x, k))) for x in data.dna]
    vocab = gen_vocab(k)

    features = np.zeros((len(data.label), len(vocab)))
    for row in range (len(features)):
        for kmer in kmers[row].keys():
            features[row][vocab[kmer]] = kmers[row][kmer]

    return labels, features

###Cross validation and Test/train split functions

In [3]:
def cross_validation_accuracy(clf, X, labels, split):
    """ 
    Compute the average testing accuracy over k folds of cross-validation. 
    Params:
        clf......A classifier.
        X........A matrix of features.
        labels...The true labels for each instance in X
        split........The fold indices
    Returns:
        The average testing accuracy of the classifier
        over each fold of cross-validation.
    """
    scores = []
    for train_index, test_index in split:
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        clf.fit(X_train, y_train)
        scores.append(accuracy_score(y_test, clf.predict(X_test)))

    return np.mean(scores)


def test_train_split(clf, split):
    """
    Compute the accuracy of a train/test split
    Params:
        clf......A classifier.
        split....indices
    Returns:
        The testing accuracy and the confusion
        matrix.
    """
    X_train, X_test, y_train, y_test = split
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    cm = confusion_matrix(y_test, y_pred)
    return score, cm

###Fit and score each model

In [4]:
def classify_all(labels, features, clfs, folds, model_names):
    """ 
    Compute the average testing accuracy over k folds of cross-validation. 
    Params:
        labels.......The true labels for each instance in X
        features.....The feature vectors for each instance
        clfs.........The classifiers to fit and test
        folds........Number of folds for cross validation
        model_names..Readable names of each classifier
    Returns:
        A dataframe with each models results
    """
    tts_split = train_test_split(
        features, labels, test_size=0.2, random_state=0)

    cv_split = KFold(len(labels), n_folds=folds, shuffle=True)

    results = pd.DataFrame(columns=["Model", "CV Score", "Test Score", "Time"])

    for x in range(len(clfs)):
        start = time.time()
        clf = clfs[x]
        cv_score = cross_validation_accuracy(clf, features, labels, cv_split)
        tts_score, cm = test_train_split(clf, tts_split)
        end = time.time()
        elapsed = end-start
        results.loc[results.shape[0]] = ([model_names[x], cv_score, tts_score, elapsed])
    return results

###Declare models

In [5]:
def main():
    file = "data/ref.100ec.pgf.seqs.filter"
    k = 3
    labels, features = featurize_data(file, k)
    folds = 5
    clfs = [SVC(), MultinomialNB(), LogisticRegression(), RandomForestClassifier(n_jobs=2), AdaBoostClassifier(n_estimators=10)]
    model_names = ["SVC", "Naive bayes", "Logistic Regression", "Random Forest", "AdaBoost"]

    print classify_all(labels, features, clfs, folds, model_names)

In [6]:
main()

                 Model  CV Score  Test Score        Time
0                  SVC  0.193299    0.188698  207.802181
1          Naive bayes  0.076903    0.071981    0.123263
2  Logistic Regression  0.288435    0.301043  147.581321
3        Random Forest  0.358339    0.352842    3.099816
4             AdaBoost  0.053085    0.054827    2.935549
